# Hashing

## 1. SHA256 Hashing Function

In [14]:
import hashlib

class SHA256Hash:
    def __init__(self, value: str):
        self.value: str = value
        self.hash: str = hashlib.sha256(value.encode('utf-8')).hexdigest()

print(SHA256Hash('test').hash)

9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08


Expand the class to include inputing a precalculated hash value.

In [15]:
from typing import Optional

class SHA256Hash:
    def __init__(self, value: str, hash: Optional[str]):
        self.value: str = value
        if hash is None:
            self.hash: str = self.hash_str(value)
        else:
            self.hash: str = hash

    @staticmethod
    def hash_str(value: str) -> str:
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

print(SHA256Hash('test', None).hash)  # 9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08
print(SHA256Hash('test', 'test').hash + f'❌')  # hash is not recalculated and potentially wrong

9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08
test❌


Add a hash checking method to verify the hash and value match and an equality method to compare two SHA256Hash objects.

In [16]:
class SHA256Hash:
    def __init__(self, value: str, hash: Optional[str] = None):
        self.value: str = value
        if hash is None:
            self.hash: str = self.hash_str(value)
        else:
            if self.verify_hash(hash):
                self.hash: str = hash
            else:
                raise ValueError(f'Hash {hash} does not match value {value}')

    @staticmethod
    def hash_str(value: str) -> str:
        return hashlib.sha256(value.encode('utf-8')).hexdigest()
    
    @staticmethod
    def verify_hash(hash: str) -> bool:
        assert isinstance(hash, str), 'hash must be a string'

        if len(hash) != 64:
            return False
        
        try:
            int(hash, 16)
        except ValueError:
            return False
        
        return True

    def __eq__(self, other: 'SHA256Hash') -> bool:
        return self.hash == other.hash
    
print(SHA256Hash('test', None).hash)  # 9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08
try:
    sha256 = SHA256Hash('test', 'test')
except ValueError:
    print('ValueError raised')

9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08
ValueError raised


Add a method to return the hash as bytes.  Also a method to add two hash together.

In [17]:
class SHA256Hash:
    def __init__(self, value: str, hash: Optional[str] = None):
        self.value = value
        self.hash = None
        if hash is None:
            self.hash = self.hash_str(value)
        else:
            if self.verify_hash(hash):
                self.hash = hash

    @staticmethod
    def hash_str(value: str) -> str:
        return hashlib.sha256(value.encode('utf-8')).hexdigest()
    
    @staticmethod
    def verify_hash(hash: str) -> bool:
        assert isinstance(hash, str), 'hash must be a string'
        assert len(hash) == 64, 'hash must be 64 characters long'
        
        return True
    
    def to_bytes(self) -> bytes:
        return bytes.fromhex(self.hash)
    
    def __add__(self, other: 'SHA256Hash') -> 'SHA256Hash':
        return SHA256Hash(value=self.value + other.value, hash=self.hash_str(self.hash + other.hash))

    def __eq__(self, other: 'SHA256Hash') -> bool:
        return self.hash == other.hash
    
print(SHA256Hash('test', None).to_bytes()) 
# b'\x9f\x86\xd0\x81\x88L}e\x9a/\xea\xa0\xc5Z\xd0\x15\xa3\xbfO\x1b+\x0b\x82,\xd1]l\x15\xb0\xf0\n\x08'

hash1 = SHA256Hash('test1', None)
hash2 = SHA256Hash('test2', None)
hash3 = hash1 + hash2
print(hash3.value + ' ' + hash3.hash)

b'\x9f\x86\xd0\x81\x88L}e\x9a/\xea\xa0\xc5Z\xd0\x15\xa3\xbfO\x1b+\x0b\x82,\xd1]l\x15\xb0\xf0\n\x08'
test1test2 2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51


# Merkle Tree

## 2. Leaf Nodes

Create a LeafNode class that inherits from the SHA256Hash class.  The LeafNode class should have a value and a hash.

In [18]:
class Leaf(SHA256Hash):
    pass


leaf = Leaf('test')
print(leaf.value + ' ' + leaf.hash)

test 9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08


## 3. Tree Leaves Class

In [19]:
class Leaves:
    def __init__(self, leaves: tuple[SHA256Hash | Leaf, ...]):
        self.leaves: tuple[SHA256Hash | Leaf, ...] = leaves

    def __str__(self):
        return '\n'.join([' '.join([leaf.value + ' ' + leaf.hash]) for leaf in self.leaves])

    def __repr__(self):
        return str(self)
    
leaves = Leaves((SHA256Hash('test1', None), SHA256Hash('test2', None)))
print(leaves)

test1 1b4f0e9851971998e732078544c96b36c3d01cedf7caa332359d6f1d83567014
test2 60303ae22b998861bce3b28f33eec1be758a213c86c93c076dbe9f558c11c752


## 4. The Merkle Tree Class

In [20]:
class MerkleTree:
    def __init__(self, base_leaves: Leaves):
        self.base_leaves: Leaves = base_leaves
        self.levels: tuple[Leaves, ...] = self.build()

    def __str__(self):
        return '\n'.join([str(leaves) for leaves in self.levels])
    
    def __repr__(self):
        return str(self)
    
    @staticmethod
    def next_level(leaves: Leaves) -> Leaves:
        if len(leaves.leaves) == 1:
            return leaves
        new_leaves: list[Leaves] = []
        for i in range(0, len(leaves.leaves), 2):
            if i + 1 < len(leaves.leaves):
                new_leaves.append(leaves.leaves[i] + leaves.leaves[i + 1])
            else:
                new_leaves.append(leaves.leaves[i])
        return Leaves(tuple(new_leaves))
    
    def build(self):
        self.levels = (self.base_leaves, )
        while len(self.levels[-1].leaves) > 1:
            self.levels += (MerkleTree.next_level(self.levels[-1]), )
        return self.levels
    
    @property
    def root(self) -> SHA256Hash:
        return self.levels[-1].leaves[0].hash
    

level = MerkleTree(leaves)
print(level)
print(MerkleTree.next_level(leaves))
next_level = MerkleTree.next_level(leaves)
print(MerkleTree.next_level(next_level))
print(level.root)

leaves_two = Leaves((SHA256Hash('test1', None), SHA256Hash('test2', None), SHA256Hash('test3', None)))
new_tree = MerkleTree(leaves_two)
print(new_tree)
print(new_tree.root)

test1 1b4f0e9851971998e732078544c96b36c3d01cedf7caa332359d6f1d83567014
test2 60303ae22b998861bce3b28f33eec1be758a213c86c93c076dbe9f558c11c752
test1test2 2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51
test1test2 2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51
test1test2 2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51
2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51
test1 1b4f0e9851971998e732078544c96b36c3d01cedf7caa332359d6f1d83567014
test2 60303ae22b998861bce3b28f33eec1be758a213c86c93c076dbe9f558c11c752
test3 fd61a03af4f77d870fc21e05e7e80678095c92d808cfb3b5c279ee04c74aca13
test1test2 2f297f1520dfd4d6a9b680536568fd3aad16a8c2d7067b654ea06dd931bccd51
test3 fd61a03af4f77d870fc21e05e7e80678095c92d808cfb3b5c279ee04c74aca13
test1test2test3 e7865859f083e773e2cb504ea8f75d959295d29522e6591006d15213acc1f820
e7865859f083e773e2cb504ea8f75d959295d29522e6591006d15213acc1f820


## Python packages for merkle trees

### pymerkle not working

In [21]:
# pip install pymerkle

In [22]:
# import pymerkle

### PySHA3 and merkletooks do not work properly.  

merkletools relies on outdated package pysha3.  sha3 is built into latest hashlib.

In [23]:
# pip install --upgrade pip setuptools wheel
# pip install pysha3 merkletools

In [24]:
# import merkletools

# mt = merkletools.MerkleTools()

### Merkly

In [25]:
# pip install merkly

In [26]:
from merkly.mtree import MerkleTree
from typing import Callable


sha256_hash_funciton: Callable[[str], str] = lambda x, y: str(hashlib.sha256(x.encode('utf-8') + y.encode('utf-8')).hexdigest())


mtree = MerkleTree(['test1', 'test2', 'test3'], sha256_hash_funciton)
print(mtree.root)
print(mtree.leafs)

e7865859f083e773e2cb504ea8f75d959295d29522e6591006d15213acc1f820
['1b4f0e9851971998e732078544c96b36c3d01cedf7caa332359d6f1d83567014', '60303ae22b998861bce3b28f33eec1be758a213c86c93c076dbe9f558c11c752', 'fd61a03af4f77d870fc21e05e7e80678095c92d808cfb3b5c279ee04c74aca13']
